# Optimisation PhysAE avec validation et réutilisation des données

Ce carnet illustre un flux complet pour contrôler la cohérence des fichiers YAML, préparer un environnement d'entraînement réutilisable, effectuer un entraînement rapide sur les phases A et B2 puis lancer une recherche d'hyperparamètres avec Optuna.

> ⚙️ Les exemples utilisent volontairement de petits jeux de données et quelques époques pour rester rapides. Adaptez les paramètres (tailles, nombres d'époques, etc.) selon vos besoins.

In [ ]:
from physae.config_loader import load_data_config, load_stage_config
from physae.validation import validate_data_config, validate_stage_config
from physae.factory import prepare_training_environment
from physae.training import train_stage_A, train_stage_B2
from physae.optimization import optimise_stage


In [ ]:
# Vérification rapide des fichiers YAML
data_cfg = load_data_config()
data_report = validate_data_config(data_cfg, name="configuration données")
print(data_report)
for stage in ("A", "B1", "B2"):
    report = validate_stage_config(load_stage_config(stage), name=f"stage {stage}")
    print(report)


In [ ]:
# Construction d'un environnement réutilisable avec un jeu réduit pour des essais rapides
data_overrides = {
    'n_train': 256,
    'n_val': 64,
    'batch_size': 8,
}
env = prepare_training_environment(config_overrides=data_overrides)
env.summary()


In [ ]:
# Entraînement rapide de la phase A
model = env.spawn_model()
train_stage_A(
    model,
    env.train_loader,
    env.val_loader,
    config_overrides={'epochs': 5},
    enable_progress_bar=True,
)


In [ ]:
# Phase B2 sur le même environnement (les poids de la phase A sont réutilisés)
train_stage_B2(
    model,
    env.train_loader,
    env.val_loader,
    config_overrides={'epochs': 5},
    enable_progress_bar=True,
)


In [ ]:
# Recherche d'hyperparamètres Optuna sur la phase A
study = optimise_stage(
    'A',
    n_trials=3,
    metric='val_loss',
    data_overrides=data_overrides,
    stage_overrides={'epochs': 5},
    reuse_dataloaders=True,
    show_progress_bar=True,
)
print('Meilleurs hyperparamètres:', study.best_params)
print('Score associé:', study.best_value)
for trial in study.trials:
    metrics = trial.user_attrs.get('metrics', {})
    val_loss = metrics.get('val_loss')
    print(f"Trial {trial.number}: val_loss={val_loss} params={trial.params}")


> 💡 Grâce à `prepare_training_environment` les DataLoader sont partagés entre les différentes étapes et entre les essais Optuna, ce qui réduit nettement le coût de préparation. Utilisez `env.spawn_model()` pour obtenir un nouveau réseau lorsque vous souhaitez recommencer un entraînement depuis zéro.